# Data Cleaning and Feature Engineering

In [1]:
import pandas as pd

df=pd.read_csv('Terry_Stops.csv')
df.head()

,Subject Age Group,Subject ID,GO / SC Num,Terry Stop ID,Stop Resolution,Weapon Type,Officer ID,Officer YOB,Officer Gender,Officer Race,...,Reported Time,Initial Call Type,Final Call Type,Call Type,Officer Squad,Arrest Flag,Frisk Flag,Precinct,Sector,Beat
0,26 - 35,-1,20150000001920,36224,Field Contact,None,7595,1978,M,White,...,05:17:00,-,-,-,NaN,N,N,-,-,-
1,-,-1,20140000120677,92317,Arrest,None,7500,1984,M,Black or African American,...,11:32:00,-,-,-,SOUTH PCT 1ST W - ROBERT,N,N,South,O,O2
2,-,-1,20150000001463,28806,Field Contact,None,5670,1965,M,White,...,07:59:00,-,-,-,NaN,N,N,-,-,-
3,-,-1,20150000001516,29599,Field Contact,None,4844,1961,M,White,...,19:12:00,-,-,-,NaN,N,-,-,-,-
4,-,-1,20150000001670,32260,Field Contact,None,7539,1963,M,White,...,04:55:00,-,-,-,NaN,N,N,-,-,-


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48508 entries, 0 to 48507
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Subject Age Group         48508 non-null  object
 1   Subject ID                48508 non-null  int64 
 2   GO / SC Num               48508 non-null  int64 
 3   Terry Stop ID             48508 non-null  int64 
 4   Stop Resolution           48508 non-null  object
 5   Weapon Type               48508 non-null  object
 6   Officer ID                48508 non-null  object
 7   Officer YOB               48508 non-null  int64 
 8   Officer Gender            48508 non-null  object
 9   Officer Race              48508 non-null  object
 10  Subject Perceived Race    48508 non-null  object
 11  Subject Perceived Gender  48508 non-null  object
 12  Reported Date             48508 non-null  object
 13  Reported Time             48508 non-null  object
 14  Initial Call Type     

This data doesn't have any apparent null values (except the 'Officer Squad' column) but just by looking at the first five rows, it is apparent that this dataset is using '-' to represent null values. Because of this, it is necessary to investigate value counts for each column individually.

In [3]:
#First drop ID columns that are not useful for prediction
df.drop(columns=['GO / SC Num', 'Terry Stop ID'], inplace=True)

#### Subject Age Group

Subjects are binned into perceived age groups. Because of the small number of null values, I will remove these rows from the dataset.

In [4]:
df['Subject Age Group'].value_counts()

26 - 35         16165
36 - 45         10342
18 - 25          9672
46 - 55          6275
56 and Above     2468
1 - 17           1990
-                1596
Name: Subject Age Group, dtype: int64

In [5]:
df = df[df['Subject Age Group'] != '-']

#### Subject ID

This column indicates whether or not the subject presented identification to the officer. If ID was presented, the subject is identified according to their ID number in the police database. If ID is not presented, this value is recorded as -1. For simplicity, I will convert this column to boolean--True (1) indicating that the subject presented identification, and False (0) meaning they did not.

In [6]:
df['Subject ID'].value_counts()

-1              33511
 7753260438        21
 15595726686       19
 8753759694        14
 7731717691        13
                ...  
 17610425747        1
 7733982605         1
 25803761039        1
 7729407367         1
 19740561406        1
Name: Subject ID, Length: 9644, dtype: int64

In [7]:
df.loc[df['Subject ID'] > 0, 'Subject ID'] = 1

In [8]:
df.loc[df['Subject ID'] < 0, 'Subject ID'] = 0

#### Weapon Type

This column has several repeated categories -- I will attempt to group all like items together to decrease noise in the model.

In [9]:
df['Weapon Type'].value_counts()

None                                    31601
-                                       12476
Lethal Cutting Instrument                1456
Knife/Cutting/Stabbing Instrument         694
Handgun                                   287
Firearm Other                              93
Blunt Object/Striking Implement            91
Club, Blackjack, Brass Knuckles            48
Firearm                                    44
Mace/Pepper Spray                          31
Other Firearm                              26
Firearm (unk type)                         15
Taser/Stun Gun                             10
None/Not Applicable                         9
Club                                        9
Fire/Incendiary Device                      7
Rifle                                       6
Shotgun                                     3
Automatic Handgun                           2
Personal Weapons (hands, feet, etc.)        2
Blackjack                                   1
Brass Knuckles                    

In [10]:
df['Weapon Type'].replace('-', 'None', inplace=True)
df['Weapon Type'].replace('None/Not Applicable', 'None', inplace=True)
df['Weapon Type'].replace('Firearm', 'Firearm Other', inplace=True)
df['Weapon Type'].replace('Other Firearm', 'Firearm Other', inplace=True)
df['Weapon Type'].replace('Firearm (unk type)', 'Firearm Other', inplace=True)
df['Weapon Type'].replace('Brass Knuckles', 'Club, Blackjack, Brass Knuckles', inplace=True)
df['Weapon Type'].replace('Blackjack', 'Club, Blackjack, Brass Knuckles', inplace=True)
df['Weapon Type'].replace('Club', 'Club, Blackjack, Brass Knuckles', inplace=True)
df['Weapon Type'].replace('Automatic Handgun', 'Handgun', inplace=True)

In [11]:
df['Weapon Type'].value_counts()

None                                    44086
Lethal Cutting Instrument                1456
Knife/Cutting/Stabbing Instrument         694
Handgun                                   289
Firearm Other                             178
Blunt Object/Striking Implement            91
Club, Blackjack, Brass Knuckles            59
Mace/Pepper Spray                          31
Taser/Stun Gun                             10
Fire/Incendiary Device                      7
Rifle                                       6
Shotgun                                     3
Personal Weapons (hands, feet, etc.)        2
Name: Weapon Type, dtype: int64

Weapon categories have been reduced. Because the categories are so unbalanced, I may try to reduce these categories further in the future, depending on the significance for the model.

#### Officer YOB

I chose to bin the data based on generation to simplify categorization.

In [12]:
df['Officer YOB'].value_counts()

1986    3280
1987    2969
1984    2703
1991    2671
1992    2456
1985    2445
1990    2295
1988    2135
1989    2026
1982    1837
1983    1685
1979    1498
1993    1498
1981    1417
1995    1272
1971    1184
1978    1145
1976    1007
1977     998
1994     920
1973     906
1980     814
1996     706
1967     705
1968     590
1970     564
1969     549
1974     535
1975     523
1997     472
1962     449
1964     432
1972     414
1965     412
1963     236
1958     215
1961     206
1966     180
1959     167
1960     128
1998      57
1954      44
1957      43
1953      33
1900      31
1955      21
1956      17
1948      10
1949       5
1952       4
1946       2
1951       1
Name: Officer YOB, dtype: int64

In [13]:
df['Officer Age Group'] = pd.cut(df['Officer YOB'],[1899, 1945, 1964, 1980, 1997, 2000], labels = ['Pre-Boomer','Baby Boomer', 'Gen X', 'Millenial', 'Gen Z'])

In [14]:
df['Officer Age Group'].value_counts()

Millenial      32787
Gen X          12024
Baby Boomer     2013
Gen Z             57
Pre-Boomer        31
Name: Officer Age Group, dtype: int64

#### Officer Race, Subject Perceived Race, and Subject Perceived Gender

Each of these features are using multiple responses to classify unknown or not reported data. For each feature, I have condensed categories to use a single category for unknown values.

In [15]:
df['Officer Race'].value_counts()

White                            35429
Two or More Races                 2715
Hispanic or Latino                2701
Asian                             2020
Black or African American         1803
Not Specified                     1460
Nat Hawaiian/Oth Pac Islander      440
American Indian/Alaska Native      313
Unknown                             31
Name: Officer Race, dtype: int64

In [16]:
df['Officer Race'].replace('Unknown', 'Not Specified', inplace=True)

In [17]:
df['Subject Perceived Race'].value_counts()

White                                        23273
Black or African American                    14062
Unknown                                       2506
Hispanic                                      1659
Asian                                         1558
-                                             1471
American Indian or Alaska Native              1365
Multi-Racial                                   801
Other                                          150
Native Hawaiian or Other Pacific Islander       67
Name: Subject Perceived Race, dtype: int64

In [18]:
df['Subject Perceived Race'].replace('-', 'Unknown', inplace=True)

In [19]:
df['Subject Perceived Gender'].value_counts()

Male                                                         37210
Female                                                        9553
Unable to Determine                                            108
Unknown                                                         19
-                                                               16
Gender Diverse (gender non-conforming and/or transgender)        6
Name: Subject Perceived Gender, dtype: int64

In [20]:
df['Subject Perceived Gender'].replace('-', 'Unknown', inplace=True)

#### Reported Date

Using this column I engineer several features -- year, day of the week, and month -- to help track trends over time.

In [21]:
#convert column to datetime to allow date-based calculations
df['Reported Date'] = pd.to_datetime(df['Reported Date'])

In [22]:
#create day of week column and map numeric values to day names
day_map={0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday',5: 'Saturday', 6: 'Sunday'} 
df['Day of Week']=df['Reported Date'].dt.weekday.map(day_map)

#create year column
df['Year'] = df['Reported Date'].dt.year

#create month column and map numeric values to month names
month_map = {1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June', 
             7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'}
df['Month'] = df['Reported Date'].dt.month.map(month_map)

#### Reported Time

The time that the incident report was filed, not the time of the stop. This column will be dropped.

In [23]:
df.drop(columns=['Reported Time'], inplace=True)

#### Initial Call Type, Final Call Type, Call Type

Initial Call Type and Final Call Type have lots of unique values, so they will be dropped. 

In [24]:
df['Initial Call Type'].value_counts()

-                                                 12773
SUSPICIOUS STOP - OFFICER INITIATED ONVIEW         3160
SUSPICIOUS PERSON, VEHICLE OR INCIDENT             3010
DISTURBANCE, MISCELLANEOUS/OTHER                   2420
ASLT - IP/JO - WITH OR W/O WPNS (NO SHOOTINGS)     2035
                                                  ...  
PROPERTY - DAMAGE (GUN RELATED)                       1
MISSING - (ALZHEIMER, ENDANGERED, ELDERLY)            1
SICK PERSON                                           1
HARBOR - WATER EMERGENCIES                            1
WARRANT PICKUP - FROM OTHER AGENCY                    1
Name: Initial Call Type, Length: 167, dtype: int64

In [25]:
df['Final Call Type'].value_counts()

-                                                     12773
--SUSPICIOUS CIRCUM. - SUSPICIOUS PERSON               3766
--PROWLER - TRESPASS                                   3333
--DISTURBANCE - OTHER                                  2694
--ASSAULTS, OTHER                                      2316
                                                      ...  
ORDER - VIOLATING DV COURT ORDER                          1
BIAS -IP/JO - RACIAL, POLITICAL, SEXUAL MOTIVATION        1
ESCAPE - PRISONER                                         1
PROWLER                                                   1
THEFT OF SERVICES                                         1
Name: Final Call Type, Length: 208, dtype: int64

In [26]:
df['Call Type'].value_counts()

911                              21400
-                                12773
ONVIEW                            9072
TELEPHONE OTHER, NOT 911          3311
ALARM CALL (NOT POLICE ALARM)      346
TEXT MESSAGE                         9
SCHEDULED EVENT (RECURRING)          1
Name: Call Type, dtype: int64

In [27]:
df['Call Type'].replace('-', 'UNKNOWN', inplace=True)
df.drop(columns = ['Initial Call Type', 'Final Call Type'], inplace=True)

#### Officer Squad

This column seems to largely overlap with Precinct and will be dropped.

In [28]:
df['Officer Squad'].value_counts()

TRAINING - FIELD TRAINING SQUAD                        5039
WEST PCT 1ST W - DAVID/MARY                            1570
WEST PCT 2ND W - D/M RELIEF                             998
SOUTHWEST PCT 2ND W - FRANK                             985
WEST PCT 1ST W - KING/QUEEN                             881
                                                       ... 
COMMUNITY OUTREACH - YOUTH VIOLENCE -SCHOOLS DETAIL       1
HR - BLEA - ACADEMY RECRUITS                              1
ZOLD CRIME ANALYSIS UNIT - ANALYSTS                       1
BURG/THEFT/JUV - NORTH                                    1
CANINE - NIGHT SQUAD                                      1
Name: Officer Squad, Length: 167, dtype: int64

In [29]:
df.drop(columns=['Officer Squad'], inplace=True)

#### Frisk Flag

A few rows have null values and will be dropped.

In [30]:
df['Frisk Flag'].value_counts()

N    35876
Y    10641
-      395
Name: Frisk Flag, dtype: int64

In [31]:
df = df[df['Frisk Flag'] != '-']

#### Precinct, Sector, and Beat

For now, I will use all three values, although based on the number of values and the number of null values, I am uncertain whether Sector and Beat will end up being useful. Precinct has a couple of data issues to resolve.

In [32]:
df['Precinct'].value_counts()

West         11498
North        10254
-             9428
East          6158
South         5551
Southwest     2220
SouthWest     1171
Unknown        181
OOJ             35
FK ERROR        21
Name: Precinct, dtype: int64

In [33]:
df['Precinct'].replace('-', 'Unknown', inplace=True)
df['Precinct'].replace('SouthWest', 'Southwest', inplace=True)

In [34]:
df['Sector'].value_counts()

-         9620
E         2247
M         2200
N         2109
K         1804
K         1702
B         1616
L         1565
D         1473
R         1376
F         1330
M         1322
S         1286
U         1247
D         1205
O         1096
J         1089
G         1052
C          996
E          961
Q          939
W          889
Q          849
N          738
F          722
R          658
O          639
B          559
S          497
G          485
U          465
L          462
W          448
C          416
J          404
99          51
Name: Sector, dtype: int64

In [35]:
df['Beat'].value_counts()

-         9556
N3        1131
E2        1050
K3        1009
M2         836
          ... 
J2          76
99          51
99          35
OOJ         23
S            2
Name: Beat, Length: 107, dtype: int64

#### Review Updated Data

At this point, I believe the data is clean enough to begin the base model.

In [36]:
df.head()

,Subject Age Group,Subject ID,Stop Resolution,Weapon Type,Officer ID,Officer YOB,Officer Gender,Officer Race,Subject Perceived Race,Subject Perceived Gender,...,Call Type,Arrest Flag,Frisk Flag,Precinct,Sector,Beat,Officer Age Group,Day of Week,Year,Month
0,26 - 35,0,Field Contact,None,7595,1978,M,White,Asian,Male,...,UNKNOWN,N,N,Unknown,-,-,Gen X,Friday,2015,April
1218,1 - 17,0,Field Contact,None,7726,1990,M,White,White,Female,...,UNKNOWN,N,N,Unknown,-,-,Millenial,Sunday,2015,May
1598,1 - 17,0,Arrest,None,7715,1992,M,White,American Indian or Alaska Native,Male,...,UNKNOWN,N,Y,East,G,G1,Millenial,Sunday,2015,September
1599,1 - 17,0,Field Contact,None,7745,1988,F,Not Specified,Unknown,Male,...,UNKNOWN,N,Y,Unknown,-,-,Millenial,Sunday,2015,April
1602,1 - 17,0,Field Contact,None,7642,1988,M,White,White,Male,...,UNKNOWN,N,N,Unknown,-,-,Millenial,Monday,2015,April


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46517 entries, 0 to 48507
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Subject Age Group         46517 non-null  object        
 1   Subject ID                46517 non-null  int64         
 2   Stop Resolution           46517 non-null  object        
 3   Weapon Type               46517 non-null  object        
 4   Officer ID                46517 non-null  object        
 5   Officer YOB               46517 non-null  int64         
 6   Officer Gender            46517 non-null  object        
 7   Officer Race              46517 non-null  object        
 8   Subject Perceived Race    46517 non-null  object        
 9   Subject Perceived Gender  46517 non-null  object        
 10  Reported Date             46517 non-null  datetime64[ns]
 11  Call Type                 46517 non-null  object        
 12  Arrest Flag       

In [38]:
#save updated data to csv file
df.to_csv('Cleaned Data')